# 🧠 Entrena tu propio LLM con datos de tu empresa
Este notebook te permite:
- Subir tus datos en formato JSON
- Fine-tunar un modelo base (Llama 3)
- Guardarlo en formato GGUF compatible con LM Studio

In [ ]:
# 🛠️ Paso 1: Instalar dependencias
!pip install torch transformers datasets peft trl accelerate bitsandbytes ctransformers lm_dataformat

In [ ]:
# 📁 Paso 2: Subir tu archivo JSON con los datos de tu empresa
from google.colab import files

print("Sube tu archivo 'datos_empresa.json'")
uploaded = files.upload()

In [ ]:
# 🧪 Paso 3: Cargar modelo base (Llama 3 optimizado)
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

model_name = "unsloth/llama-3-8b-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

In [ ]:
# 📊 Paso 4: Preparar los datos
from datasets import load_dataset

dataset = load_dataset('json', data_files='datos_empresa.json')

def format_data(examples):
    inputs = [f"### Pregunta:\n{q}\n\n### Respuesta:\n{a}" for q, a in zip(examples["input"], examples["output"])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["output"], max_length=256, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized_datasets = dataset.map(format_data, batched=True)

In [ ]:
# 🔧 Paso 5: Configurar fine-tuning con LoRA
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

In [ ]:
# 🏋️‍♂️ Paso 6: Entrenamiento
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./modelo_empresa",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=3e-4,
    logging_steps=10,
    num_train_epochs=3,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

print("🚀 Iniciando entrenamiento...")
trainer.train()

In [ ]:
# 💾 Paso 7: Guardar modelo ajustado
model.save_pretrained("./modelo_empresa_final")
tokenizer.save_pretrained("./modelo_empresa_final")

In [ ]:
# 🧱 Paso 8: Convertir a GGUF (formato LM Studio)
from ctransformers import convert

convert("./modelo_empresa_final", "./modelo_empresa_gguf", model_type="llama")

In [ ]:
# 📥 Paso 9: Comprimir y descargar el modelo
!zip -r modelo_empresa_gguf.zip ./modelo_empresa_gguf

from google.colab import files
print("📦 Descargando modelo ajustado:")
files.download("modelo_empresa_gguf.zip")